# Using super and the MRO to make things work for you.

I've defined Base and SuperBase in `helpers.py`, so they can be imported other classes we will redefine. 

Base for our purpose is just `object` that prints its own name. We expect to see it so we know when the very bottom of certain paths gets hit.

## Multiple inheritance with multiple arguments

The previous examples didn't give arguments to their `__init__` lets correct that.

In [ ]:
from helpers import Base, SuperBase

In [ ]:
class ChildA(SuperBase):
    def __init__(self, A):
        print('Enter ChildA.__init__')
        self.a = A
        super().__init__()
        print('Exit ChildA.__init__')
    
    def my_arg(self):
        print(f'ChildA.my_arg: {self.a}')

class ChildB(ChildA):
    def __init__(self, A, B):
        print('Enter ChildB.__init__')
        self.b = B
        super().__init__(A)
        print('Exit ChildB.__init__')

    def my_arg(self):
        print(f'ChildB.my_arg: {self.b}')

my_class = ChildB(1, 2)
my_class.my_arg()

During that process which we could have continued 'turtles all the way down' we made ChildB inherit ChildA. Child B has attributes a and b, but the method my_arg uses only that defined by ChildB. Enter super.

In [ ]:
class ChildC(ChildA):
    def __init__(self, A, C):
        print('Enter ChildC.__init__')
        self.c = C
        super().__init__(A)
        print('Exit ChildC.__init__')

    def my_arg(self):
        print(f'ChildC.my_arg: {self.c}')
        super().my_arg()

my_class = ChildC(1, 3)
my_class.my_arg()

The ability to run that code was not lost just tucked away in a super, a super just looks for the next definition of the requested function name in the MRO.

In fact in the `__init__` method the number of arguments is being whittled away from 2 to 1 but you can do this quite creatively a complex class with lots of `__init__` arguments may quite quickly split them up into multiple inherited `__init__`.

In [ ]:
# The super looks for the next definition of the method in the MRO
# We shouldn't but could get weird with that.

class ChildD(ChildA):
    def __init__(self, A, D):
        print('Enter ChildD.__init__')
        self.d = D
        self._a = A
        print('Exit ChildD.__init__')

    def my_arg(self):
        print(f'ChildD.my_arg: {self.d}')
        super().__init__(self._a)
        super().my_arg()

my_class = ChildD(1, 4)
my_class.my_arg()

### That was grim, and confusing.

Let's make it worse!

In [ ]:
class ChildE(ChildA):
    def __init__(self, A, E):
        print('Enter ChildE.__init__')
        self.e = E
        super().__init__(A)
        print('Exit ChildE.__init__')

class ChildF(ChildE):
    def __init__(self, A, E, F):
        print('Enter ChildF.__init__')
        self.f = F
        super().__init__(A, E)
        print('Exit ChildF.__init__')
    
    def my_arg(self):
        print(f'ChildF.my_arg: {self.f}')
        super().my_arg()

my_class = ChildF(1, 5, 6)
my_class.my_arg()

print(f"ChildF MRO:{ChildF.__mro__}")

## Wait, what?

`super` just jumped child E and went to A I thought it was accessing the parent!?

No super searches MRO in order until it finds that method. If you are familiar with `PATH` resolution then you can intuit this the same way. If you are unfamiliar with `PATH` resolution you just learnt two things!

# Get back to multiple inheritance please.

Let's start with a Mixin

In [ ]:
class MixinA:
    def my_name(self):
        print("Name:")
        print(f"__name__:{self.__class__.__name__}")
    
    def all_my_names(self):
        print("All Names:")
        for i in self.__class__.__mro__:
            print(f"__name__:{i.__name__}")
        

In [ ]:
class ChildG(ChildA, MixinA):
    def __init__(self, A, G):
        print('Enter ChildG.__init__')
        self.g = G
        super().__init__(A)
        print('Exit ChildG.__init__')

    def my_arg(self):
        print(f'ChildG.my_arg: {self.g}')
        super().my_arg()

my_class = ChildG(1, 7)
my_class.my_arg()
my_class.my_name()
my_class.all_my_names()

The next step is likely figuring out how the correct input variables make it to the correct `__init__` methods.

Starting with a design lets say we want this:

![MI1.png](figures/MI1.png)

Which is nearly, just remove child H, as simple as we can possibly make a MI structure. We will, for now ignore the fact that classes have methods.

In [ ]:
class NewBase(object):
    def __init__(self, A, B):
        print('Enter NewBase.__init__')
        self.a = A
        self.b = B
        super().__init__()
        print('Exit NewBase.__init__')

class ChildH(NewBase):
    def __init__(self, A, B, C):
        print('Enter ChildH.__init__')
        self.c = C
        super().__init__(A, B)
        print('Exit ChildH.__init__')

class SomeOtherClass(object):
    def __init__(self, E, F):
        print('Enter SomeOtherClass.__init__')
        self.e = E
        self.f = F
        print('Exit SomeOtherClass.__init__')

class ChildI(ChildH, SomeOtherClass):
    def __init__(self, A, B, C, D, E, F):
        print('Enter ChildI.__init__')
        # The issue here is what to pass to an init method such that each super get the right arguments
        # super().__init__(A, B, C, D, E, F) # This will fail ChildH doesn't take D, E, and F
        super().__init__(A, B, C) # This will fail as the super at the base, excluding object, of the MRO doesn't get E and F
        self.d = D
        print('Exit ChildI.__init__')

print(f"ChildI MRO: {ChildI.__mro__}")
my_class = ChildI(1, 2, 3, 4, 5, 6)


ChildI MRO: (<class '__main__.ChildI'>, <class '__main__.ChildH'>, <class '__main__.NewBase'>, <class '__main__.SomeOtherClass'>, <class 'object'>)
Enter ChildI.__init__
Enter ChildH.__init__
Enter NewBase.__init__


TypeError: SomeOtherClass.__init__() missing 2 required positional arguments: 'E' and 'F'

## We need make the active choice in ALL our classes that multiple inheritance is what we are doing

This means making some adaptations to all classes. 

### OR

We can use some other class directly.

### But it still isnt working quite right

```python

class ChildI(ChildH, SomeOtherClass):
    def __init__(self, A, B, C, D, E, F):
        print('Enter ChildI.__init__')
        # The issue here is what to pass to an init method such that each super get the right arguments
        # super().__init__(A, B, C, D, E, F) # This will fail ChildH doesn't take D, E, and F
        super().__init__(A, B, C) # This line still fails due to the super in NewBase calling the __init__ of SomeOtherClass.
        SomeOtherClass().__init__(E, F) # This would solve all our problems IF our base didn't have a super.
        self.d = D
        print('Exit ChildI.__init__')
```

Problem is, our base class for all the reasons discusses already SHOULD HAVE a super. Making it be a purposeful blocker of multiple inheritance is limiting to people who may want to use our code. We can do it but why tell others what to do when we could make our code more sustainable and more interoperable, more people using == more citations?

## Active choice to make this Multiple Inheritance friendly.

This is going to require the use of **kwargs

### Aside on `**kwargs`

*If you don't know about **kwargs (and *args) then you should not be messing with multiple inheritance. Come back in 6 months of intensive Python use or a couple years of normal use. Where in both cases you have been thinking about 'why Python works' not just 'how to make it work for me'.*

Quick refresher, **kwargs is commonly used to pass keyword arguments to a function as a dictionary. We can do **banana, but we don't because conformity is good for everyone. We can do fun things like.

```python
def getting_kwargy(a, b=1, **kwargs):
    print(a)
    print(b)
    print(kwargs)


my_kwargs = {
    'a': 10,
    'b': 2,
    'c': 42
}
getting_kwargy(**my_kwargs)

my_other_kwargs = {
    'c': 42,
    'd': 1
}

getting_kwargy(10, 2, **my_other_kwargs)
```

You should be able to parse what happens here. 
The take away is that python allows arguments to be defined positionally, delivered by keyword, or vice versa.
Then we can use `**kwargs` to capture any leftovers preventing issues with providing too many arguments.
Ignoring completely `*args` here because we are not going to use them.

### Adding `**kwargs` to our structure

We will attempt the above structure again but this time making sure our `__init__` passes unused variables down the call, when we get to the last `__init__` MRO call **kwargs is empty.
Here we have `SomeOtherClass` that is _intentionally_ poorly written so will end the `__init__` chain and has no kwargs so this is where the `TypeError: <class>.__init__() got an unexpected keyword argument '<var>'` error is created. 

```python
class SomeOtherClass(object):
    def __init__(self, E, F, **kwargs):
        print('Enter SomeOtherClass.__init__')
        super().__init__(**kwargs)
        self.e = E
        self.f = F
        print('Exit SomeOtherClass.__init__')
```

 If `SomeOtherClass` were correctly implemented as above then `object` would raise `TypeError: object.__init__() takes exactly one argument (the instance to initialize)`.

 This actually is a great example of why all classes should have a `super().__init__(**kwargs)` it makes that error appear if you pass the wrong stuff to the constructor.

 ### See the diagram/code below for our new structure


![MI2.png](figures/MI2.png)

In [ ]:
class MIBase(object):
    def __init__(self, A, B, **kwargs):
        print('Enter MIBase.__init__')
        self.a = A
        self.b = B
        super().__init__(**kwargs)
        print('Exit MIBase.__init__')

class ChildJ(MIBase):
    def __init__(self, C, **kwargs):
        print('Enter ChildH.__init__')
        self.c = C
        super().__init__(**kwargs)
        print('Exit ChildH.__init__')

# Unchanged from above, commented here for reference
"""
class SomeOtherClass(object):
    def __init__(self, E, F):
        print('Enter SomeOtherClass.__init__')
        self.e = E
        self.f = F
        print('Exit SomeOtherClass.__init__')
"""

class ChildK(ChildJ, SomeOtherClass):
    def __init__(self, D, **kwargs):
        """
        # A solid docstring here to enable good documentation of expected arguments
        Args:
            D: int
            A: int (keyword only)
            B: int (keyword only)
            E: int (keyword only)
            F: int (keyword only)
        """
        print('Enter ChildI.__init__')
        # The issue here is what to pass to an init method such that each super get the right arguments
        # super().__init__(A, B, C, D, E, F) # This will fail ChildH doesn't take D, E, and F
        super().__init__(**kwargs) # This will fail as the super at the base, excluding object, of the MRO doesn't get E and F
        self.d = D
        print('Exit ChildI.__init__')

print(f"ChildI MRO: {ChildI.__mro__}")
my_class = ChildK(4, A=1, B=2, C=3, E=4, F=5)
print("\nAnd again\n")
my_class_2 = ChildK(D=4, C=3, E=4, F=5, A=1, B=2)
print("\nAnd again\n")
class_args = {
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
    'E': 5,
    'F': 6
} 
my_class_3 = ChildK(**class_args)
print("\nFinally\n")
my_args_core = {
    'A': 1,
    'B': 2,
    'C': 3
}
my_args_other = {
    'E': 5,
    'F': 6
}
my_class_4 = ChildK(D=4, **my_args_core, **my_args_other)

ChildI MRO: (<class '__main__.ChildI'>, <class '__main__.ChildH'>, <class '__main__.NewBase'>, <class '__main__.SomeOtherClass'>, <class 'object'>)
Enter ChildI.__init__
Enter ChildH.__init__
Enter NewBase.__init__
Enter SomeOtherClass.__init__
Exit SomeOtherClass.__init__
Exit NewBase.__init__
Exit ChildH.__init__
Exit ChildI.__init__

And again

Enter ChildI.__init__
Enter ChildH.__init__
Enter NewBase.__init__
Enter SomeOtherClass.__init__
Exit SomeOtherClass.__init__
Exit NewBase.__init__
Exit ChildH.__init__
Exit ChildI.__init__

And again

Enter ChildI.__init__
Enter ChildH.__init__
Enter NewBase.__init__
Enter SomeOtherClass.__init__
Exit SomeOtherClass.__init__
Exit NewBase.__init__
Exit ChildH.__init__
Exit ChildI.__init__

Finally

Enter ChildI.__init__
Enter ChildH.__init__
Enter NewBase.__init__
Enter SomeOtherClass.__init__
Exit SomeOtherClass.__init__
Exit NewBase.__init__
Exit ChildH.__init__
Exit ChildI.__init__


# Finally!
Not only have we successfully built a multiple inheritance structure with an `__init__` that sets up every single class in the structure it does it flexibly allowing keywords, dicts, and positional arguments for the last child. There is nothing stopping us making it work with all positional arguments.

## Full MI with *args

This example makes it work regardless of what you do. However we need to change our docstring up.

![MI2.png](figures/MI3.png)

In [ ]:
class MIBase2(object):
    def __init__(self, A, B, *args, **kwargs):
        print('Enter MI2Base.__init__')
        self.a = A
        self.b = B
        super().__init__(*args, **kwargs)
        print('Exit MI2Base.__init__')

class ChildL(MIBase2):
    def __init__(self, C, *args, **kwargs):
        print('Enter ChildL.__init__')
        self.c = C
        super().__init__(*args, **kwargs)
        print('Exit ChildL.__init__')

# Unchanged from above, commented here for reference
"""
class SomeOtherClass(object):
    def __init__(self, E, F):
        print('Enter SomeOtherClass.__init__')
        self.e = E
        self.f = F
        print('Exit SomeOtherClass.__init__')
"""

class ChildM(ChildL, SomeOtherClass):
    def __init__(self, D, *args, **kwargs):
        """
        # A solid docstring here to enable 
        # good documentation of expected arguments
        Args:
            D: int
            A: int
            B: int
            E: int
            F: int
        """
        print('Enter ChildM.__init__')
        super().__init__(*args, **kwargs)
        self.d = D
        print('Exit ChildM.__init__')

print(f"ChildI MRO: {ChildI.__mro__}")
my_class = ChildM(4, 1, 2, 3, 5, 6)
print("\nAnd again\n")
my_class_2 = ChildM(4, *(1, 2, 3), **{'E': 5, 'F': 6})

ChildI MRO: (<class '__main__.ChildI'>, <class '__main__.ChildH'>, <class '__main__.NewBase'>, <class '__main__.SomeOtherClass'>, <class 'object'>)
Enter ChildI.__init__
Enter ChildH.__init__
Enter NewBase.__init__
Enter SomeOtherClass.__init__
Exit SomeOtherClass.__init__
Exit NewBase.__init__
Exit ChildH.__init__
Exit ChildI.__init__

And again

Enter ChildI.__init__
Enter ChildH.__init__
Enter NewBase.__init__
Enter SomeOtherClass.__init__
Exit SomeOtherClass.__init__
Exit NewBase.__init__
Exit ChildH.__init__
Exit ChildI.__init__


This now takes all arguments in any form, positional arguments need to be in the correct position and it's up to you to communicate this via _very_ good documentation both in the code via docstrings and in the published docs.

## Applying this further

As we learnt `__init__` is just one function that uses super.
We can use a similar pattern for other `super`ed functions, but this is a good 'exercise to the reader'.